In [3]:
pip install geocoder


     |████████████████████████████████| 102kB 3.8MB/s 
Note: you may need to restart the kernel to use updated packages.


In [4]:
#importing Libraries

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder# to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [5]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [6]:

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [7]:
neighborhoodList = []

In [8]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [9]:
mum_df=pd.DataFrame({'neighbourhood':neighborhoodList})

In [11]:
mum_df

,neighbourhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup
5,Borivali
6,Charkop
7,Chembur
8,Dahisar
9,Devipada


In [12]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [15]:
coords = [ get_latlng(neighborhood) for neighborhood in mum_df["neighbourhood"].tolist() ]

In [17]:
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00534722389655, 72.85580272012932],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21198153260436, 72.83757275783374],
 [19.048530000000028, 72.93222000000003

In [18]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [19]:
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

In [20]:
mum_df.head()

,neighbourhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560


In [21]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [23]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  

In [24]:
map_mum

In [25]:
CLIENT_ID = 'UGUNFLNZSUSHHRAM3OOLRZKH5CQOCOOEVBWR2S0CKOFNK0MG' # your Foursquare ID
CLIENT_SECRET = 'J0DFWKQWYJ0FNUZ2FCELNON0K2VDNMLA0NGSQG0TXOHJA1HT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [26]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
dataframe = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Worli Sea Face,Scenic Lookout,Worli,Khan Abdul Gaffar Khan Road,19.009216,72.815022,"[{'label': 'display', 'lat': 19.00921649707565...",278,400018,IN,Mumbai,Mahārāshtra,India,"[Worli (Khan Abdul Gaffar Khan Road), Mumbai 4...",NaN,4b7fecb5f964a520c04430e3
1,SMAAASH,Athletics & Sports,"SMAAASH, Kamala Mills, Gate No. 4, Lower Pare...",Lower Parel,19.004472,72.826114,"[{'label': 'display', 'lat': 19.00447181182, '...",1026,400013,IN,Mumbai,Mahārāshtra,India,"[SMAAASH, Kamala Mills, Gate No. 4, Lower Par...",NaN,50715ea7e4b0db7c162854e8
2,Bombay Canteen,Indian Restaurant,Kamla Mills,Lower Parel,19.003410,72.827816,"[{'label': 'display', 'lat': 19.00341017043938...",1235,400013,IN,Mumbai,Mahārāshtra,India,"[Kamla Mills (Lower Parel), Mumbai 400013, Mah...",NaN,54d39713498e901bd9f9e6eb
3,Naturals,Ice Cream Shop,"Rustom Court, HP Petrol Pump Building, Next To...","RG Thadani Marg, Flora Hotel Road, Worli",19.001954,72.814966,"[{'label': 'display', 'lat': 19.00195395239303...",643,400018,IN,Mumbai,Mahārāshtra,India,"[Rustom Court, HP Petrol Pump Building, Next T...",NaN,4f748916e4b02c4c6d551b27
4,Slink & Bardot,French Restaurant,Thadani Mansion,Worli Village,19.017170,72.817532,"[{'label': 'display', 'lat': 19.01717044484786...",1085,400030,IN,Mumbai,Mahārāshtra,India,"[Thadani Mansion (Worli Village), Mumbai 40003...",NaN,591d41b58134885f2cd56960
5,"And Chillies, Pan Asian Cuisine",Thai Restaurant,PB Marg; Near Paragon Centre;,"Worli, Lower Parel",19.006001,72.824465,"[{'label': 'display', 'lat': 19.00600121247706...",814,400067,IN,Mumbai,Mahārāshtra,India,"[PB Marg; Near Paragon Centre; (Worli, Lower P...",NaN,4e784013fa769112d53b7e6d
6,Tamasha,Lounge,NaN,NaN,19.006162,72.827481,"[{'label': 'display', 'lat': 19.00616218312774...",1124,NaN,IN,NaN,NaN,India,[India],NaN,5880f1455a58695417a8138d
7,Jai Hind Lunch Home,Seafood Restaurant,Tulsipipe Road,Dadar West,19.002183,72.829512,"[{'label': 'display', 'lat': 19.00218252696195...",1452,NaN,IN,Mumbai,Mahārāshtra,India,"[Tulsipipe Road (Dadar West), Mumbai, Mahārāsh...",NaN,4bfe4e1855539c7488d2bcf3
8,Starbucks Coffee: A Tata Alliance,Coffee Shop,Century Bhavan,NaN,19.012851,72.823886,"[{'label': 'display', 'lat': 19.01285087827859...",952,400030,IN,Mumbai,Mahārāshtra,India,"[Century Bhavan, Mumbai 400030, Mahārāshtra, I...",NaN,53ae4241498ed1411b8d2597
9,Koko's Kitchen,Asian Restaurant,Koko,Kamala Mills,19.002569,72.827672,"[{'label': 'display', 'lat': 19.00256928015480...",1258,NaN,IN,Mumbai,Mahārāshtra,India,"[Koko (Kamala Mills), Mumbai, Mahārāshtra, India]",NaN,578e485c498ef438f27439ee


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
mum_venues = getNearbyVenues(names=mum_df['neighbourhood'],
                                   latitudes=mum_df['Latitude'],
                                   longitudes=mum_df['Longitude']
                                  )

Andheri
Anushakti Nagar
Baiganwadi
Bandra
Bhandup
Borivali
Charkop
Chembur
Dahisar
Devipada
Dombivli
Eastern Suburbs (Mumbai)
Ghatkopar
Goregaon
Grant Road
Jogeshwari
Juhu
Kalyan
Kandivali
Kanjurmarg
Kausa
Kurla
Mahavir Nagar (Kandivali)
Mankhurd
Matharpacady, Mumbai
Mira Road
Mogra Village
Mulund
Mumbra
Pestom sagar
Seven Bungalows
Shil Phata
Sion, Mumbai
Sonapur, Bhandup
Thakur village
Tilak Nagar (Mumbai)
Uttan
Vashi
Vikhroli
Wadala
Western Suburbs (Mumbai)
Worli


In [34]:
mum_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,McDonald's,19.119691,72.846102,Fast Food Restaurant
3,Andheri,19.118459,72.841763,Cafe Alfa,19.119667,72.843560,Indian Restaurant
4,Andheri,19.118459,72.841763,Shoppers Stop,19.114935,72.842708,Department Store


In [35]:
mum_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Andheri,18,18,18,18,18,18
Anushakti Nagar,4,4,4,4,4,4
Bandra,14,14,14,14,14,14
Bhandup,5,5,5,5,5,5
Borivali,17,17,17,17,17,17
Charkop,5,5,5,5,5,5
Chembur,23,23,23,23,23,23
Dahisar,9,9,9,9,9,9
Devipada,4,4,4,4,4,4


In [36]:
mum_venues['Venue Category'].unique()

array(['Bakery', 'Indian Restaurant', 'Fast Food Restaurant',
       'Department Store', 'Vegetarian / Vegan Restaurant', 'Restaurant',
       'Athletics & Sports', 'Food Court', 'Chinese Restaurant',
       'Bookstore', 'Electronics Store', 'Pet Store', 'Accessories Store',
       'Café', 'Burger Joint', 'Bus Station', 'Convenience Store',
       'Arcade', 'Train Station', 'Paper / Office Supplies Store',
       'Italian Restaurant', 'Platform', 'Pier', 'Furniture / Home Store',
       'Asian Restaurant', 'Food Truck', 'Ice Cream Shop',
       'Clothing Store', 'Snack Place', 'Shop & Service', 'Pizza Place',
       'Shopping Mall', 'Playground', 'Motorcycle Shop', 'Bar', 'Gym',
       'Lounge', 'Garden', 'Seafood Restaurant', 'Gym / Fitness Center',
       'South Indian Restaurant', 'Kids Store', 'Whisky Bar', 'Bank',
       'Dog Run', 'Lighthouse', 'Smoke Shop', 'Pharmacy', 'Pool Hall',
       'Record Shop', 'Donut Shop', 'Coffee Shop', 'Juice Bar',
       'Mobile Phone Shop', 'Bed &

In [37]:
mumbai_onehot = pd.get_dummies(mum_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mum_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Women's Store,ATM,Accessories Store,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Creperie,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Furniture / Home Store,Garden,Gastropub,General Entertainment,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lighthouse,Lounge,Market,Men's Store,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multiplex,Music Venue,Neighborhood,Nightclub,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Pool Hall,Pub,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()


In [39]:
mumbai_grouped.head()

,Neighborhood,Women's Store,ATM,Accessories Store,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Creperie,Cupcake Shop,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Furniture / Home Store,Garden,Gastropub,General Entertainment,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lighthouse,Lounge,Market,Men's Store,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multiplex,Music Venue,Nightclub,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Pool Hall,Pub,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar
0,Andheri,0.0,0.0,0.055556,0.0,0.00,0.0,0.0,0.055556,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.055556,0.00,0.0,0.055556,0.0,0.055556,0.000000,0.0,0.0,0.000000,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.055556,0.0,0.0,0.0,0.0,0.055556,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.055556,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.055556,0.0
1,Anushakti Nagar,0.0,0.0,0.000000,0.0,0.25,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.25,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,Bandra,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.142857,0.0,0.000000,0.000000,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.357143,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.000000,0.0,0.071429,0.000000,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.000000,0.0
3,Bhandup,0.0,0.0,0.000000,0.0,0.00,0.0,0.2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.200000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.400000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.000000,0.0
4,Borivali,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.058824,0.00,0.0,0.058824,0.0,0.117647,0.176471,0.0,0.0,0.000000,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0

In [41]:
mum_restro = mumbai_grouped[["Neighborhood","Asian Restaurant","BBQ Joint","Bistro","Chinese Restaurant","Fast Food Restaurant","Gastropub","Indian Restaurant","Italian Restaurant","Japanese Restaurant","Mughlai Restaurant","Restaurant","Seafood Restaurant","South Indian Restaurant","Sushi Restaurant","Steakhouse","Thai Restaurant","Vegetarian / Vegan Restaurant"]]

In [42]:
mum_restro.head()

,Neighborhood,Asian Restaurant,BBQ Joint,Bistro,Chinese Restaurant,Fast Food Restaurant,Gastropub,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mughlai Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Sushi Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Andheri,0.0,0.0,0.0,0.055556,0.055556,0.0,0.166667,0.000000,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.055556
1,Anushakti Nagar,0.0,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,Bandra,0.0,0.0,0.0,0.000000,0.000000,0.0,0.357143,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,Bhandup,0.2,0.0,0.0,0.000000,0.400000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,Borivali,0.0,0.0,0.0,0.117647,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.058824


In [60]:
kclusters = 3

mum_clustering = mum_restro.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 2, 1, 1, 0, 1, 1, 1], dtype=int32)

In [61]:
mum_merged = mum_restro.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [63]:
mum_merged.rename(columns={"Neighborhood": "neighbourhood"}, inplace=True)
mum_merged.head()

,neighbourhood,Asian Restaurant,BBQ Joint,Bistro,Chinese Restaurant,Fast Food Restaurant,Gastropub,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mughlai Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Sushi Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels
0,Andheri,0.0,0.0,0.0,0.055556,0.055556,0.0,0.166667,0.000000,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.055556,0
1,Anushakti Nagar,0.0,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,2
2,Bandra,0.0,0.0,0.0,0.000000,0.000000,0.0,0.357143,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0
3,Bhandup,0.2,0.0,0.0,0.000000,0.400000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,2
4,Borivali,0.0,0.0,0.0,0.117647,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.058824,1


In [64]:
mum_merged = mum_merged.join(mum_df.set_index("neighbourhood"), on="neighbourhood")

print(mum_merged.shape)
mum_merged.head()
mum_merged

(39, 21)


,neighbourhood,Asian Restaurant,BBQ Joint,Bistro,Chinese Restaurant,Fast Food Restaurant,Gastropub,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mughlai Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Sushi Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.166667,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00000,0.000000,0.055556,0,19.118459,72.841763
1,Anushakti Nagar,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,2,19.042830,72.927340
2,Bandra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.357143,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0,19.054370,72.840170
3,Bhandup,0.200000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,2,19.145560,72.948560
4,Borivali,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.00000,0.000000,0.058824,1,19.229360,72.857510
5,Charkop,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1,19.208660,72.826120
6,Chembur,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.217391,0.000000,0.000000,0.000000,0.086957,0.043478,0.043478,0.000000,0.00000,0.000000,0.086957,0,19.062180,72.902410
7,Dahisar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.222222,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1,19.250030,72.859070
8,Devipada,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1,19.224690,72.866050
9,Dombivli,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1,19.212750,73.083240


In [65]:
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

,neighbourhood,Asian Restaurant,BBQ Joint,Bistro,Chinese Restaurant,Fast Food Restaurant,Gastropub,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mughlai Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Sushi Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.166667,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00000,0.000000,0.055556,0,19.118459,72.841763
24,Mira Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0,19.280032,72.867932
2,Bandra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.357143,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0,19.054370,72.840170
34,Vashi,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.357143,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.00000,0.000000,0.071429,0,19.084650,72.904810
18,Kanjurmarg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0,19.131380,72.935680
25,Mulund,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.133333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.00000,0.000000,0.066667,0,19.171830,72.955650
6,Chembur,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.217391,0.000000,0.000000,0.000000,0.086957,0.043478,0.043478,0.000000,0.00000,0.000000,0.086957,0,19.062180,72.902410
30,"Sonapur, Bhandup",0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.210526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.105263,0.000000,0,19.163940,72.935440
29,"Sion, Mumbai",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.00000,0.000000,0.041667,0,19.043590,72.864120
23,"Matharpacady, Mumbai",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.230769,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.00000,0.000000,0.000000,0,19.044920,72.867205


In [66]:
import numpy as np
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['neighbourhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [67]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,neighbourhood,Asian Restaurant,BBQ Joint,Bistro,Chinese Restaurant,Fast Food Restaurant,Gastropub,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mughlai Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Sushi Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0.000000,0.0,0.055556,0.055556,0.000000,0.166667,0.000000,0.0,0.000000,0.055556,0.000000,0.000000,0.0,0.0,0.000000,0.055556,0,19.118459,72.841763
24,Mira Road,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0,19.280032,72.867932
2,Bandra,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.357143,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0,19.054370,72.840170
34,Vashi,0.000000,0.000000,0.0,0.000000,0.071429,0.000000,0.357143,0.000000,0.0,0.000000,0.071429,0.000000,0.000000,0.0,0.0,0.000000,0.071429,0,19.084650,72.904810
18,Kanjurmarg,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0,19.131380,72.935680
25,Mulund,0.066667,0.000000,0.0,0.000000,0.066667,0.000000,0.133333,0.000000,0.0,0.000000,0.066667,0.000000,0.000000,0.0,0.0,0.000000,0.066667,0,19.171830,72.955650
6,Chembur,0.043478,0.000000,0.0,0.000000,0.000000,0.000000,0.217391,0.000000,0.0,0.000000,0.086957,0.043478,0.043478,0.0,0.0,0.000000,0.086957,0,19.062180,72.902410
30,"Sonapur, Bhandup",0.000000,0.000000,0.0,0.000000,0.105263,0.000000,0.210526,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.105263,0.000000,0,19.163940,72.935440
29,"Sion, Mumbai",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.125000,0.000000,0.0,0.0,0.000000,0.041667,0,19.043590,72.864120
23,"Matharpacady, Mumbai",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.230769,0.000000,0.0,0.000000,0.000000,0.153846,0.000000,0.0,0.0,0.000000,0.000000,0,19.044920,72.867205


In [68]:

mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,neighbourhood,Asian Restaurant,BBQ Joint,Bistro,Chinese Restaurant,Fast Food Restaurant,Gastropub,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mughlai Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Sushi Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
26,Mumbra,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,1,19.190540,73.022660
28,Seven Bungalows,0.025641,0.0,0.025641,0.051282,0.000000,0.000000,0.076923,0.000000,0.000000,0.0,0.000000,0.051282,0.025641,0.000000,0.00000,0.0,0.000000,1,19.131342,72.816342
32,Tilak Nagar (Mumbai),0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,1,18.996160,72.852790
33,Uttan,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,1,26.866340,80.938840
36,Wadala,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,1,19.017200,72.858170
31,Thakur village,0.031250,0.0,0.000000,0.031250,0.062500,0.000000,0.062500,0.000000,0.031250,0.0,0.031250,0.000000,0.000000,0.000000,0.03125,0.0,0.000000,1,19.210200,72.875410
22,Mankhurd,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,1,19.048530,72.932220
19,Kausa,0.037037,0.0,0.000000,0.037037,0.000000,0.000000,0.037037,0.000000,0.037037,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.074074,1,19.127580,72.825390
20,Kurla,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,1,19.064980,72.880690
37,Western Suburbs (Mumbai),0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,1,19.197010,72.827680


In [69]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,neighbourhood,Asian Restaurant,BBQ Joint,Bistro,Chinese Restaurant,Fast Food Restaurant,Gastropub,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mughlai Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Sushi Restaurant,Steakhouse,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
17,Kandivali,0.0,0.0,0.0,0.076923,0.153846,0.0,0.000,0.153846,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,2,19.211982,72.837573
27,Pestom sagar,0.0,0.0,0.0,0.000000,0.125000,0.0,0.125,0.125000,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,2,19.070640,72.902170
3,Bhandup,0.2,0.0,0.0,0.000000,0.400000,0.0,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,2,19.145560,72.948560
35,Vikhroli,0.0,0.0,0.0,0.000000,0.200000,0.0,0.000,0.000000,0.0,0.0,0.000,0.2,0.0,0.0,0.0,0.0,0.0,2,19.111090,72.927810
1,Anushakti Nagar,0.0,0.0,0.0,0.000000,0.250000,0.0,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,2,19.042830,72.927340
21,Mahavir Nagar (Kandivali),0.0,0.0,0.0,0.076923,0.153846,0.0,0.000,0.153846,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,2,19.211982,72.837573


### Observations:
The major restaurant types in most parts are Indian,Chinese,Fastfood and Italian restaurants. Certain restaurant types are present more in some areas than in others. For Example, Juhu in Cluster 0 has more Gastropubs than other parts of Mumbai. Therefore, by using this clustering output, restaurant owners/ entrepreneurs can decide which type of restaurant to open such that they will face the least competition. Many suburbs of Mumbai do not have an authentic Indian restaurant. If Indian restaurants are opened in these suburbs, they will face a much lower competition from other restaurants. Certain suburbs of Mumbai that are more upmarket can open Gastropubs,Bistros,BBQ Joints,Steakhouses,Japanese and Sushi restaurants that can cater to the upmarket crowd. Another lucrative option would be opening a Vegan/Vegetarian restaurant in many restaurant in many suburbs. 

As for the location, neighborhoods in Cluster 2 are recommended.Various Restaurant types are not present here and will thus face far less competition as compared to clusters 0 and 1.